In [1]:
# Importing the Libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Device setup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using Device :',device) 

Using Device : cuda


In [3]:
# Data Transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))   # Mean, std for RGB
])


# Load CIFAR10 Dataset 
train_dataset = datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transform)

# Dataloader 
train_dataloader = DataLoader(train_dataset, shuffle=True, num_workers=8, batch_size=64)
test_dataloader = DataLoader(test_dataset, shuffle=True, num_workers=8, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
# Define Model

# class SimpleCNN(nn.Module):
#     def __init__(self):
#         super(SimpleCNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # input=3x32x32 -> output=32x32x32
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # -> 64x32x32
#         self.pool = nn.MaxPool2d(2, 2)  # -> 64x16x16
#         self.fc1 = nn.Linear(64 * 16 * 16, 128)
#         self.fc2 = nn.Linear(128, 10)   # CIFAR10 has 10 classes

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = self.pool(x)
#         x = x.view(-1, 64 * 16 * 16)    # Flatten
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

# model = SimpleCNN().to(device)
# model.eval()



class BetterCNN(nn.Module):
    def __init__(self):
        super(BetterCNN, self).__init__()
        # Conv Block 1
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)  # 32x32 -> 16x16

        # Conv Block 2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2, 2) # 16x16 -> 8x8

        # Fully connected layers
        self.fc1 = nn.Linear(128*8*8, 256)
        self.dropout = nn.Dropout(0.5)   # Dropout to avoid overfitting
        self.fc2 = nn.Linear(256, 10)    # CIFAR10 has 10 classes

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)

        x = x.view(-1, 128*8*8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


model = BetterCNN().to(device)
model.eval()

BetterCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=256, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_feature

In [18]:
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training Loop
epochs= 5
for epoch in range(epochs):
    model.train()         # Mode switch in pytorch, not actually training itself
                          # Some layers in NN behave differently during training 
                          # Dropout. BatchNOrm, so .train() tells Switch all model into training mode
    running_loss = 0.0
    for idx, (images, labels) in enumerate(train_dataloader):
        images, labels = images.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Batch [{idx}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch+1}/{epochs}], Avg Loss: {running_loss/len(train_dataloader):.4f}")

Epoch [1/5], Batch [0], Loss: 1.1274
Epoch [1/5], Batch [100], Loss: 1.4657
Epoch [1/5], Batch [200], Loss: 1.5543
Epoch [1/5], Batch [300], Loss: 1.6957
Epoch [1/5], Batch [400], Loss: 1.1592
Epoch [1/5], Batch [500], Loss: 1.1922
Epoch [1/5], Batch [600], Loss: 0.9466
Epoch [1/5], Batch [700], Loss: 0.9042
Epoch [1/5], Avg Loss: 1.2387
Epoch [2/5], Batch [0], Loss: 1.1014
Epoch [2/5], Batch [100], Loss: 1.0629
Epoch [2/5], Batch [200], Loss: 0.7973
Epoch [2/5], Batch [300], Loss: 0.8103
Epoch [2/5], Batch [400], Loss: 1.0974
Epoch [2/5], Batch [500], Loss: 1.0841
Epoch [2/5], Batch [600], Loss: 0.8505
Epoch [2/5], Batch [700], Loss: 1.0276
Epoch [2/5], Avg Loss: 0.9557
Epoch [3/5], Batch [0], Loss: 0.9536
Epoch [3/5], Batch [100], Loss: 0.8618
Epoch [3/5], Batch [200], Loss: 1.1510
Epoch [3/5], Batch [300], Loss: 0.9572
Epoch [3/5], Batch [400], Loss: 0.8048
Epoch [3/5], Batch [500], Loss: 0.7616
Epoch [3/5], Batch [600], Loss: 0.7259
Epoch [3/5], Batch [700], Loss: 0.8174
Epoch [3/5

In [19]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on CIFAR-10 test set: {100 * correct / total:.2f}%")

Accuracy on CIFAR-10 test set: 77.49%
